# Neural Network creation

This script takes results from "aggregated results", and generates neural
networks based on them.


In [ ]:
import sys
from utils.setup import Setup

argv           = sys.argv[1:]
# argv           = ['-c', 'cfg_single_lon120.yml']
argv = ["-c", "cfg_testing.yml"]

setup = Setup(argv)


# Create Neural Networks

Rasp et al.'s CBRAIN `fc_model` is used here for reference. As we are
doing a simpler version, I've opted to create our own, shorter function
for NN creation.

In [ ]:
from tensorflow.keras import Sequential, Input
from tensorflow.keras.layers import Dense


def dense_nn(input_shape, output_shape, hidden_layers, activation):
    """
    Creates a dense NN in base of the parameters received
    """
    model = Sequential()
    model.add(Input(shape=input_shape))

    for n_layer_nodes in hidden_layers:
        model.add(Dense(n_layer_nodes, activation=activation))

    model.add(Dense(output_shape))
    return model


def fc_model(
    input_shape,
    output_shape,
    hidden_layers,
    activation,
    conservation_layer=False,
    inp_sub=None,
    inp_div=None,
    norm_q=None,
):
    inp = Input(shape=(input_shape,))

    # First hidden layer
    x = Dense(hidden_layers[0])(inp)
    x = act_layer(activation)(x)

    # Remaining hidden layers
    for h in hidden_layers[1:]:
        x = Dense(h)(x)
        x = act_layer(activation)(x)

    if conservation_layer:
        x = SurRadLayer(inp_sub, inp_div, norm_q)([inp, x])
        x = MassConsLayer(inp_sub, inp_div, norm_q)([inp, x])
        out = EntConsLayer(inp_sub, inp_div, norm_q)([inp, x])

    else:
        out = Dense(output_shape)(x)

    return tf.keras.models.Model(inp, out)


In [ ]:
from utils.variable import Variable

In [ ]:
import numpy as np
import tensorflow as tf
from pathlib import Path

# # NOTE: This will generate a lot of models, it may be better to put
# # them on different folders
# MODEL_FILENAME_PATTERN = "model-{variable}-a{pc_alpha}-t{threshold}.h5"


class ModelDescription:
    """
    Object that stores a Keras model and metainformation about it.
    
    Attributes
    ----------
    output : Variable
        Output variable of the model.
    pc_alpha : str
        Meta information. PC alpha used to find the parents.
    threshold : str
        Meta information. Gridpoint threshold used to select the parents.
    parents : list(Variable)
        List of the variables (and variable level) that cause the output
        variable.
    hidden_layers : list(int)
        Description of the hidden dense layers of the model
        (default [32, 32, 32]).
    activation : Keras-compatible activation function
        Activation function used for the hidden dense layers
        (default "relu").
    model : Keras model
        Model created using the given information.
        See `_build_model()`.
    input_vars_dict:
    
    #TODO
    
    """
    
    def __init__(
        self,
        output,
        parents,
        model_type,
        pc_alpha,
        threshold,
        setup
    ):
        """
        Parameters
        ----------
        output : str
            Output variable of the model in string format. See Variable.
        parents : list(str)
            List of strings for the variables that cause the output variable.
            See Variable.
        model_type : str
            # TODO
        pc_alpha : str
            Meta information. PC alpha used to find the parents.
        threshold : str
            Meta information. Gridpoint threshold used to select the parents.
        hidden_layers : list(int)
            Description of the hidden dense layers of the model.
        activation : Keras-compatible activation function
            Activation function used for the hidden dense layers.
        """
        self.setup = setup
        self.output = Variable.parse_var_name(output)
        parents = [Variable.parse_var_name(p) for p in parents]
        self.parents = sorted(parents, key=lambda x: self.setup.input_order_list.index(x))
        self.model_type = model_type
        self.pc_alpha = pc_alpha
        self.threshold = threshold
        self.model = self._build_model()
        self.input_vars_dict = ModelDescription._build_vars_dict(self.parents)
        self.output_vars_dict = ModelDescription._build_vars_dict([self.output])

    def _build_model(self):
        """
        Build a Keras model with the given information.
        
        Some parameters are not configurable, taken from Rasp et al.
        """
        input_shape = len(parents)
        input_shape = (input_shape,)
        model = dense_nn(
            input_shape=input_shape,
            output_shape=1,  # Only one output per model
            hidden_layers=self.setup.hidden_layers,
            activation=self.setup.activation,
        )
        model.compile(
            # TODO? Move to configuration
            optimizer = "adam", # From train.py (default)
            loss = "mse", # From 006_8col_pnas_exact.yml
            metrics = [tf.keras.losses.mse], # From train.py (default)
        )
        return model
    
    @staticmethod
    def _build_vars_dict(list_variables):
        """
        Convert the given list of Variable into a dictionary to be used
        on the data generator.
        
        Parameters
        ----------
        list_variables : list(Variable)
            List of variables to be converted to the dictionary format
            used by the data generator
        
        Returns
        -------
        vars_dict : dict{str : list(int)}
            Dictionary of the form {ds_name : list of levels}, where
            "ds_name" is the name of the variable as stored in the
            dataset, and "list of levels" a list containing the indices
            of the levels of that variable to use, or None for 2D
            variables.
        """
        vars_dict = dict()
        for variable in list_variables:
            ds_name = variable.var.ds_name # Name used in the dataset
            if variable.var.dimensions == 2:
                vars_dict[ds_name] = None
            elif variable.var.dimensions == 3:
                levels = vars_dict.get(ds_name, list())
                levels.append(variable.level_idx)
                vars_dict[ds_name] = levels
        return vars_dict
    
    def fit_model(self, x, validation_data, epochs, callbacks, verbose = 1):
        self.model.fit(
            x = x,
            validation_data = validation_data,
            epochs = epochs,
            callbacks = callbacks,
            verbose = verbose,
        )
    
    def get_path(self, base_path):
        # TODO? Separate "filename" from full path, to use in Tensorboard
        path = Path(base_path, self.model_type)
        if self.model_type == "CausalSingleNN":
            path = path / Path("a{pc_alpha}-t{threshold}/".format(
                pc_alpha = self.pc_alpha,
                threshold = self.threshold
            ))
        str_hl = str(self.setup.hidden_layers).replace(", ", "_")
        str_hl = str_hl.replace("[", "").replace("]", "")
        path = path / Path("hl_{hidden_layers}-act_{activation}-e_{epochs}/".format(
            hidden_layers = str_hl,
            activation = self.setup.activation,
            epochs = self.setup.epochs
        ))
        return path
    
    def get_filename(self):
        i_var = setup.output_order.index(self.output.var)
        i_level = self.output.level_idx
        if i_level is None:
            i_level = 0
        return f"{i_var}_{i_level}"
    
    def save_model(self, base_path):
        folder = self.get_path(base_path)
        filename = self.get_filename()
        print(f"Using filename {filename}.")
        # Save model
        self.model.save(Path(folder, f"{filename}_model.h5"))
        # Save weights
        self.model.save_weights(Path(folder, f"{filename}_weights.h5"))
        # Save input list
        self.save_input_list(folder, filename)
        
    def save_input_list(self, folder, filename):
        input_list = self.get_input_list()
        with open(Path(folder, f"{filename}_input_list.txt"), "w") as f:
            for line in input_list:
                print(str(line), file = f)
        
    def get_input_list(self):
        return [int(var in self.parents) for var in setup.input_order_list]

    def __str__(self):
        name = f"{self.model_type}: {self.output}"
        if self.pc_alpha != None:
            # pc_alpha and threshold should be either both None or both not None
            name += f", a{self.pc_alpha}-t{self.threshold}"
        return name
#         return self.get_filename()

    def __repr__(self):
        return repr(str(self))

    def __hash__(self):
        return hash(str(self))

    def __eq__(self, other):
        return str(self) == str(other)


## List of models

In [ ]:
model_descriptions = list()

## Read aggregated results and generate CausalSingleNN models

Currently, results are loaded from a previously created file. This was
made for testing purposes, and we may want to replace it for an analysis.

In that replacement, the idea would be to execute more or less the same
as aggregate_results up to the moment the file is currently saved, and
continue with that object (aggregated results) here.

## Generate SingleNN models

In [ ]:
if setup.do_single_nn:
    from utils.constants import SPCAM_Vars
    # TODO Only parents & children in configuration
    parents = list() # TODO Parents and levels
    for spcam_var in setup.var_parents:
        if spcam_var.dimensions == 3:
            for level, _ in setup.parents_idx_levs:
                # There's enough info to build a Variable list
                # However, it could be better to do a bigger reorganization
                var_name = f"{spcam_var.name}-{round(level, 2)}"
                parents.append(var_name)
        elif spcam_var.dimensions == 2:
            var_name = spcam_var.name
            parents.append(var_name)
    
    output_list = list()
    for spcam_var in setup.var_children:
        if spcam_var.dimensions == 3:
            for level, _ in setup.children_idx_levs:
                # There's enough info to build a Variable list
                # However, it could be better to do a bigger reorganization
                var_name = f"{spcam_var.name}-{round(level, 2)}"
        elif spcam_var.dimensions == 2:
            var_name = spcam_var.name
        output_list.append(var_name)    

    for output in output_list:
        model_description = ModelDescription(
            output,
            parents,
            "SingleNN",
            pc_alpha = None,
            threshold = None,
            setup = setup,
        )
        model_descriptions.append(model_description)


## Generate CausalSingleNN models

In [ ]:
if setup.do_causal_single_nn:
    import utils.utils as utils
    from pathlib import Path
    example_aggregated = Path(
        "./aggregated_results",
        setup.yml_filename.replace(".yml", ".obj")
    )
    # example_aggregated = "./aggregated_results/cfg_testing.obj"
    aggregated_results = utils.load_results(example_aggregated)


In [ ]:
if setup.do_causal_single_nn:
    for output, pc_alpha_dict in aggregated_results.items():
        print(output)
        if len(pc_alpha_dict) == 0:  # May be empty
            # TODO How to approach this?
            print("Empty results")
            pass
        for pc_alpha, pc_alpha_results in pc_alpha_dict.items():
            var_names = np.array(pc_alpha_results["var_names"])
            for threshold, parent_idxs in pc_alpha_results["parents"].items():
                parents = var_names[parent_idxs]
                model_description = ModelDescription(
                    output,
                    parents,
                    "CausalSingleNN",
                    pc_alpha,
                    threshold,
                    setup = setup,
                )
                model_descriptions.append(model_description)


# Training

## Data generator

In [ ]:
from pathlib import Path

from neural_networks.cbrain.data_generator import DataGenerator
from neural_networks.cbrain.utils import load_pickle

out_scale_dict = load_pickle(Path(setup.out_scale_dict_folder, setup.out_scale_dict_fn))
input_transform = (setup.input_sub, setup.input_div)

def build_train_generator(input_vars_dict, output_vars_dict):
    train_gen = DataGenerator(
        data_fn          = Path(setup.train_data_folder, setup.train_data_fn),
        input_vars_dict       = input_vars_dict,
        output_vars_dict      = output_vars_dict,
        # norm_fn          = Path(DATA_FOLDER, NORM_FN),
        norm_fn          = Path(setup.normalization_folder, setup.normalization_fn),
        input_transform  = input_transform,
        output_transform = out_scale_dict,
        batch_size       = setup.batch_size,
        shuffle          = True, # This feature doesn't seem to work
    )
    return train_gen


nlat=64
nlon=128
ngeo = nlat * nlon

def build_valid_generator(input_vars_dict, output_vars_dict):
    valid_gen = DataGenerator(
            data_fn          = Path(setup.train_data_folder, setup.valid_data_fn),
            input_vars_dict       = input_vars_dict,
            output_vars_dict      = output_vars_dict,
            norm_fn          = Path(setup.normalization_folder, setup.normalization_fn),
            input_transform  = input_transform,
            output_transform = out_scale_dict,
            batch_size       = ngeo,
            shuffle          = False,
            #xarray           = True,
    )
    return valid_gen


## Fit models

Train all models in the model list and store the results.

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping
from neural_networks.cbrain.learning_rate_schedule import LRUpdate
from neural_networks.cbrain.save_weights import save_norm

for model_description in model_descriptions:
    print(model_description)
    
    input_vars_dict = model_description.input_vars_dict
    output_vars_dict = model_description.output_vars_dict
    
    # TODO Test that the data is being taken correctly
    with build_train_generator(
        input_vars_dict, output_vars_dict
    ) as train_gen, build_valid_generator(
        input_vars_dict, output_vars_dict
    ) as valid_gen:
        lrs = LearningRateScheduler(LRUpdate(
            init_lr = setup.init_lr,
            step = setup.step_lr,
            divide = setup.divide_lr
        ))
        tensorboard = tf.keras.callbacks.TensorBoard(
            log_dir=Path(
                model_description.get_path(setup.tensorboard_folder),
                model_description.get_filename()
            ),
            histogram_freq=0,
            write_graph=True,
            write_images=False,
            update_freq="epoch",
            profile_batch=2,
            embeddings_freq=0,
            embeddings_metadata=None,
        )
        early_stop = EarlyStopping(
            monitor="val_loss",
            patience=setup.train_patience
        )
        model_description.fit_model(
            x = train_gen,
            validation_data = valid_gen,
            epochs = setup.epochs,
            callbacks = [lrs, tensorboard, early_stop],
            verbose = setup.train_verbose,
        )
        model_description.save_model(setup.nn_output_path)
        # Better to do this after saving the model to
        # avoid having to create the folder manually
        save_norm( 
            input_transform = train_gen.input_transform,
            output_transform = train_gen.output_transform, 
            save_dir = str(model_description.get_path(setup.nn_output_path)),
            filename = model_description.get_filename()
        )

#         print(len(train_gen))
